Media Cloud: Measuring Attention
================================

At this point you should be ready to query Media Cloud for data. You can use boolean query syntax - [read our query guide](https://mediacloud.org/support/query-guide) for more details about the exact syntax (it runs an [ElasticSearch search](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html) under the hood). **This notebook demonstrates how to quickly measure attention paid to an issue by the media**.

Studying media attention is critical for understanding how much readers are exposed to an issue, and has a long tradition. Media Cloud supports investigating attention within individual sources we track, or within collections or sources. We have wide global coverage with both national-level and regional/state-level collections for most countries. You can [browse our geographic collections](https://search.mediacloud.org/collections/news/geographic) to see more.

Our Python API exposes two methods that are particularly helpful for studying attention: 

* `story_count`: return the total number of stories in our database matching your query
* `story_count_over_time`: return the total number of stories in our database matching your query _by day_
* `story_list`: page through the actual stories that match your query

In [8]:
# Set up your API key and import needed things
import os, mediacloud.api
from importlib.metadata import version

# from dotenv import load_dotenv
import datetime as dt
from IPython.display import JSON
import bokeh.io

bokeh.io.reset_output()
bokeh.io.output_notebook()
import datetime
import os

import mediacloud.api
from rich import print as pp

MC_API_KEY = os.getenv("MEDIA_CLOUD_API_KEY")
if MC_API_KEY is None:
    try:
        with open("../config/media.cloud.key") as f:
            MC_API_KEY = f.read().strip()
        pp("[bold green][SUCCESS] MC API Key found.[/bold green]")
    except FileNotFoundError:
        pp(
            "[bold red][ERROR] MC API key not found. Check ENV 'MEDIA_CLOUD_API_KEY' or file './config/media.cloud.key'[/bold red]"
        )
else:
    pp("[bold green][SUCCESS] MC API Key found.[/bold green]")
search_api = mediacloud.api.SearchApi(MC_API_KEY)
f'Using Media Cloud python client v{version("mediacloud")}'

Loading BokehJS ...

[SUCCESS] MC API Key found.

'Using Media Cloud python client v4.1.4'

## Attention from a Single Media Source
You can start by looking at attention from a single media source to a topic you are interested in. We have almost a million media sources in our system, but only about 100,000 of them are ones that we regularly collect stories from, via RSS feeds or more recently from their sitemaps. You can get the internal id number for any source by searching for it in our [Directory](https://search.mediacloud.org/directory) and noting the ID number just under the large title on that page.

In [12]:
# check how many stories include the phrase "climate change" in the Washington Post (media id #2)
my_query = '"climate change"'  # note the double quotes used to indicate use of the whole phrase
start_date = dt.date(2023, 11, 1)
end_date = dt.date(2023, 11, 2)
sources = [2]
search_api.story_count(my_query, start_date, end_date, source_ids=sources)

{'relevant': 15, 'total': 425}

In [14]:
# you can see this count by day as well
results = search_api.story_count_over_time(
    my_query, start_date, end_date, source_ids=sources
)
# JSON(results)
pp(results)

[
    {'date': datetime.date(2023, 11, 1), 'total_count': 211, 'count': 7, 'ratio': 0.03317535545023697},
    {'date': datetime.date(2023, 11, 2), 'total_count': 214, 'count': 8, 'ratio': 0.037383177570093455}
]

In [15]:
# and you can chart attention over time with some simple notebook work (using Bokeh here)
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

df = pd.DataFrame.from_dict(results)
df["date"] = pd.to_datetime(df["date"])
source = ColumnDataSource(df)
p = figure(x_axis_type="datetime", width=900, height=250)
p.line(
    x="date", y="count", line_width=2, source=source
)  # your could use `ratio` instead of `count` to see normalized attention
show(p)

### Normalizing within a Source

Looking at absolute attention at the story level is intriguing, but you probably want to normalize this in some way to support comparisons between sources. To do this, we typically compare attention to the total number of stories we have from a source within that same timespan. That's why the `story_count` endpoint returns two numbers. `relevant` is the number of stories that matched your query.
 * `relevant` is the number that matched your search with all the conditions
 * `total` is the number that matched your search without the query terms

In [16]:
results = search_api.story_count(my_query, start_date, end_date)
source_ratio = results["relevant"] / results["total"]
'{:.2%} of the Washington Post stories were about "climate change"'.format(source_ratio)

'0.54% of the Washington Post stories were about "climate change"'

## Research Within a Country - using collections

[We have wide global coverage](https://sources.mediacloud.org/#/collections/country-and-state), with sources published in a country grouped into collections. For many of these countries we also have collections of media sources published in the various states and provinces. Lets compare the source-level attention to country-level attention.

In [ ]:
# check in our collection of country-level US National media sources
my_query = '"climate change"'
US_NATIONAL_COLLECTION = 34412234
results = search_api.story_count(
    my_query, start_date, end_date, collection_ids=[US_NATIONAL_COLLECTION]
)
us_country_ratio = results["relevant"] / results["total"]
'{:.2%} of stories from national-level US media sources mentioneded "climate change"'.format(
    us_country_ratio
)

In [ ]:
# now we can compare this to the source-level coverage
coverage_ratio = 1 / (source_ratio / us_country_ratio)
'"climate change" received {:.2} times less coverage in WashPo than you might expect based on other US national papers'.format(
    coverage_ratio
)

In [ ]:
# or compare to another country (India in this case)
INDIA_NATIONAL = 34412118
results = search_api.story_count(
    '"climate change"', start_date, end_date, collection_ids=[INDIA_NATIONAL]
)
india_country_ratio = results["relevant"] / results["total"]
'{:.2%} of stories from national-level Indian media sources in 2019 mentioned "climate change"'.format(
    india_country_ratio
)

In [ ]:
coverage_ratio = 1 / (india_country_ratio / us_country_ratio)
'at the national level "climate change" is covered {:.2} times less in India than the US'.format(
    coverage_ratio
)

## Listing Stories

Story counts are fine, but often what you really want is the story themselves. Note that **we cannot provide story content** due to copyright restrictions. However, you can get a list of all the URLs and then fetch them yourself. We can also return word counts down to the story level (see the "language" notebook for more info on that).

In [18]:
# grab the most recent stories about this issue
stories, _ = search_api.story_list(my_query, start_date, end_date)
stories[:3]

[{'id': 'dee7470dce016f1590d166919e2a959d1aff6e5e8105d911baf583abb3ad8025',
  'media_name': 'teachingexpertise.com',
  'media_url': 'teachingexpertise.com',
  'title': '50 Celebratory Earth Day Books For Kids',
  'publish_date': datetime.date(2023, 11, 2),
  'url': 'https://www.teachingexpertise.com/books/earth-day-books-for-kids/',
  'language': 'en',
  'indexed_date': datetime.datetime(2024, 7, 17, 7, 24, 16, 813213)},
 {'id': '81046b94a255a5a39c1b8996b95b111801de42f5fdd4e6387c89b394bbd01201',
  'media_name': 'auswaertiges-amt.de',
  'media_url': 'auswaertiges-amt.de',
  'title': 'UNIDAS: Together for women’s rights and democracy',
  'publish_date': datetime.date(2023, 11, 2),
  'url': 'https://www.auswaertiges-amt.de/en/aussenpolitik/regionaleschwerpunkte/lateinamerika/unidas-ni-una-menos/2518476',
  'language': 'en',
  'indexed_date': datetime.datetime(2024, 6, 20, 2, 15, 40, 254150)},
 {'id': '6bb05f429910b01ae4185c7d487971c061984ded44e880b80d0f8cf776058c18',
  'media_name': 'swar

If you want to list ALL the stories matching, you need to page through the results. This is accomplished via the `pagination_token` parameter. This code snippet pages through all the stories in a query.

In [ ]:
# let's fetch all the stories matching our query on one day
all_stories = []
more_stories = True
pagination_token = None
while more_stories:
    page, pagination_token = search_api.story_list(
        my_query,
        dt.date(2023, 11, 29),
        dt.date(2023, 11, 30),
        collection_ids=[US_NATIONAL_COLLECTION],
        pagination_token=pagination_token,
    )
    all_stories += page
    more_stories = pagination_token is not None
len(all_stories)

As you may have noted, this can take a while for long time periods. If you look closely you'll notice that it can't be easily parallelized, because it requires content in the results to make the next call. A workaround is to divide you query up by time and query in parallel for something like each day. This can speed up the response. Also **just contact us directly if you are trying to do larger data dumps, or hit up against your API quota**.

### Writing a CSV of Story Data

What you probably want is a csv of all this story data. Here's a quick exmaple of dumping that data to a CSV (like our Search tool does).

In [ ]:
import csv

fieldnames = [
    "id",
    "publish_date",
    "title",
    "url",
    "language",
    "media_name",
    "media_url",
    "indexed_date",
]
with open("story-list.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction="ignore")
    writer.writeheader()
    for s in all_stories:
        writer.writerow(s)

In [ ]:
# and let's make sure it worked by checking out by loading it up as a pandas DataFrame
import pandas

df = pandas.read_csv("story-list.csv")
df.head()

## Top Media Sources

Attention within a collection is useful to know, and to compare across collections. We also offer the ability to see the
media that had the most stories matching your search.

In [ ]:
# List media producing the most stories matching the search
INDIA_NATIONAL = 34412118
results = search_api.sources(
    '"climate change"', start_date, end_date, collection_ids=[INDIA_NATIONAL]
)
JSON(results)